<a href="https://colab.research.google.com/github/Tiabet/BaekJoon/blob/main/TF_KoGPT_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture

!pip install transformers
!pip install accelerate

In [2]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.13.0
Uninstalling tensorflow-2.13.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.13.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? ㅛ
Your response ('ㅛ') was not one of the expected responses: y, n, 
Proceed (Y/n)? ㅛ
Your response ('ㅛ') was not one of the expected responses: y, n, 
Proceed (Y/n)? Y
  Successfully uninstalled tensorflow-2.13.0


In [3]:
%%capture
!pip install tensorflow==2.8.0

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import torch
import tensorflow as tf

In [3]:
# Load model directly
from transformers import AutoTokenizer, TFGPT2LMHeadModel, GPT2Config

tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

# Add special tokens, including [SEP]
special_tokens = {'bos_token': '</s>', 'eos_token': '</s>', 'pad_token': '<pad>',  'unk_token' : '<unk>', 'mask_token' : '<mask>'}
tokenizer.add_special_tokens(special_tokens)

# Load the existing model checkpoint
model_config = GPT2Config.from_pretrained("Tiabet/Tiabet-TFkoGPT-complete_story-epoch-1")

# Initialize the model
model = TFGPT2LMHeadModel.from_pretrained("Tiabet/Tiabet-TFkoGPT-complete_story-epoch-1", config=model_config)

model.resize_token_embeddings(len(tokenizer))

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at Tiabet/Tiabet-TFkoGPT-complete_story-epoch-1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [4]:
import json

def jsonlload(fname):
    with open(fname, "r", encoding="utf-8") as f:
        lines = f.read().strip().split("\n")
        j_list = [json.loads(line) for line in lines]

    return j_list


def jsonldump(j_list, fname):
    with open(fname, "w", encoding='utf-8') as f:
        for json_data in j_list:
            f.write(json.dumps(json_data, ensure_ascii=False)+'\n')


In [5]:
j_list = jsonlload("/content/drive/MyDrive/nikluge-sc-2023-test.jsonl")

In [6]:
import pandas as pd
import tqdm

# Initialize empty lists to store data
sentence1_list = []
sentence3_list = []

# Read the JSONL file line by line and extract data
with open("/content/drive/MyDrive/nikluge-sc-2023-test.jsonl", 'r', encoding='utf-8') as file:
    for line in file:
        data = eval(line)  # Evaluate the JSONL line as a Python dictionary
        sentence1 = data['input']['sentence1']
        sentence3 = data['input']['sentence3']
        sentence1_list.append(sentence1)
        sentence3_list.append(sentence3)


# Create a Pandas DataFrame
test_data = pd.DataFrame({'sentence1': sentence1_list, 'sentence3': sentence3_list})

In [7]:
batch_size = 32

steps = len(test_data) // batch_size + 1

def get_test_data():
    for sentence1, sentence3 in zip(test_data.sentence1.to_list(), test_data.sentence3.to_list()):
        bos_token = [tokenizer.bos_token_id]
        sent = tokenizer.encode('첫 번째 문장 <unused1>' + sentence1 +'세 번째 문장 <unused2>' + sentence3 + '사이에 들어갈 두 번째 문장 <unused3>')
        yield bos_token + sent

test_dataset = tf.data.Dataset.from_generator(get_test_data, output_types=tf.int32)

def pad_sequence_left(sequence, max_length):
    pad_length = max_length - tf.shape(sequence)[0]
    padded_sequence = tf.pad(sequence, [[pad_length, 0]], constant_values=tokenizer.pad_token_id)
    return padded_sequence

max_sequence_length = 128

test_dataset = test_dataset.map(lambda x: pad_sequence_left(x, max_sequence_length))
test_dataset = test_dataset.batch(batch_size)



In [10]:
test_results = []

for batch in tqdm.tqdm_notebook(test_dataset, total=steps):
    output_ids = model.generate(input_ids=batch, max_length=180, do_sample=True, top_k=30, top_p=0.7)

    generated_sentences = [tokenizer.decode(ids.numpy().tolist(),skip_special_tokens = True) for ids in output_ids]

    test_results.extend(generated_sentences)

<ipython-input-10-29cc1ce0c8db>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm.tqdm_notebook(test_dataset, total=steps):


  0%|          | 0/470 [00:00<?, ?it/s]

In [15]:
test_results[1]

'첫 번째 문장  언덕 위에 있는 남성이 팔을 위로 들고 있었다.세 번째 문장  그러자 다른 사람이 제발 생쇼하지 말라고 조소했다.사이에 들어갈 두 번째 문장  남성은 그 손을 들었다.'

In [ ]:
test_results[12755 ]  ='부산외국어대학교는 입학생 정원 미달 사태를 빚었다. 인기있는 학과를 신설하자 더 이상 입학생 정원 미달 사태는 일어나지 않았다. 입학 정원 미달 사태가 터지기 전에 그 문제를 해결했다.'

In [12]:
# Define a function to extract the final sentence from a component
def extract_final_sentence(component):
    sentences = component.split('두 번째 문장  ')
    final_sentence = sentences[1]
    return final_sentence

# Use a list comprehension to apply the function to the list of components
final_sentences = [extract_final_sentence(component) for component in test_results]

In [16]:
for idx, oup in enumerate(final_sentences):
    j_list[idx]["output"] = oup

jsonldump(j_list, '/content/drive/MyDrive/Prompt_TFKoGPT_epoch1.json')

In [14]:
final_sentences[0]

'승무원이 서영이에게 기차를 잘 타냐고 물었다.'